In [0]:
import re

dbutils.widgets.text("table_name", "")
table_name = dbutils.widgets.get("table_name")

dbutils.widgets.text("year", "")
year = dbutils.widgets.get("year")

file_name = f"{table_name}-bordereaux-{year}.csv"
file_mount = f"/mnt/bordereaux/bronze/{year}"

# Check if file is in the mount, raise information, then end
if not any(file_info.name == file_name for file_info in dbutils.fs.ls(file_mount)):
    dbutils.notebook.exit(f"File {file_name} not found in {file_mount}.")
                     
excel_file_path = f"{file_mount}/{file_name}"
delta_table = f"hive_metastore.bordereaux.{table_name}"

df = spark.read.csv(
    excel_file_path,
    header=True,
    inferSchema=False,
    multiLine=True,         # In case some fields have line breaks
    quote='"',              # Properly handle comma-separated quoted fields
    escape='"',             # Handle quotes inside quoted fields
    mode="PERMISSIVE"       # Handle malformed lines gracefully
)

# Then load data in the bordereaux delta table
df.write.format("delta") \
    .mode("append") \
    .option("mergeSchema", "true") \
    .saveAsTable(delta_table)